In [2]:
from utils import load_model_and_tokenizer, update_model_with_mask_token, analyze_tokenizer

update_model_with_mask_token()

model, tokenizer = load_model_and_tokenizer()

# analyze_tokenizer(tokenizer)

Loading tokenizer...
Found 1648 musical tokens out of 51894 total
Added 1628 musical tokens to vocabulary
<MASK> token already exists in vocabulary
Loading base model with correct vocab size...
Loading LoRA adapters...
Saving updated model...
Update complete!
Found 1648 musical tokens out of 51894 total
Added 1628 musical tokens to vocabulary



---

## RETRAIN

---

In [ ]:
from retrain import train_lora
from utils import load_model_and_tokenizer

model, tokenizer = load_model_and_tokenizer(for_training = True)
metrics_history = train_lora(model, tokenizer)

Found 1648 musical tokens out of 51894 total
Trainable params: 2,162,688 / 358,662,144 (0.60%)


Tokenizing pre-tokenized dataset:   0%|          | 0/435 [00:00<?, ? examples/s]

Tokenizing pre-tokenized dataset:   0%|          | 0/37 [00:00<?, ? examples/s]

Found 13 rest tokens in musical vocabulary


Epoch 1/5 [Train]:   0%|          | 0/218 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


In [ ]:
from retrain import plot_metrics

plot_metrics(metrics_history)

: 